In [1]:
import os
import pickle as pk
import tensorflow as tf
from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import unicodedata
import re
import os
import time

import json
from glob import glob
from PIL import Image
import pickle
import pandas as pd
ACCEPTABLE_AVAILABLE_MEMORY = 8000
import subprocess as sp

os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
def mask_unused_gpus(leave_unmasked=1,random=True):
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    try:
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
        memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
        available_gpus = [i for i, x in enumerate(memory_free_values) if x > ACCEPTABLE_AVAILABLE_MEMORY]
        import numpy as np
        #print(available_gpus)
        if random:
           available_gpus = np.asarray(available_gpus)
           np.random.shuffle(available_gpus)
        if len(available_gpus) < leave_unmasked: raise ValueError('Found only %d usable GPUs in the system' %\
                                                                  len(available_gpus))
        os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, available_gpus[:leave_unmasked]))
        #print(','.join(map(str, available_gpus[:leave_unmasked])))
    except Exception as e:
        print('"nvidia-smi" is probably not installed. GPUs are not masked', e)

#mask_unused_gpus(2)
mask_unused_gpus(1)


import os
import re
import time
import datetime
import sys
sys.path.append('../src')
#sys.path.append('/content/drive/MyDrive/PerformanceNarrativesRuns/')

import pandas as pd
import seaborn as sns
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')
nltk.download('brown')
import functools

from nltk import wordpunct_tokenize,word_tokenize
# Read the dataset
full_data= json.load(open('../dataset/annotation_data_with_augmentations.json'))['data']
#jdata = json.load(open('james_submission.json'))
first_batch= json.load(open('../dataset/annotation_download.json'))
full_data = train_data = full_data+first_batch
test_data = json.load(open('../dataset/test set.json'))
classes_tokens =[f'C{i}' for i in range(1,6)]

[nltk_data] Downloading package punkt to /home/essel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /home/essel/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
from data_utils import *

In [3]:
pc= full_data[123]
examples = composePreambleAndInputs(pc,identical_metrics=identicals,augnment_metrics=True,augment_output_order=True)
examples

{'preamble': '<MetricsInfo> f1score | VALUE_LOW | 48.54% <|> recall | VALUE_MODERATE | 70.12% <|> accuracy | VALUE_MODERATE | 81.13% <|> precision | VALUE_LOW | 37.12%  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|IMBALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB'],
 'dataset_attribute': ['<|IMBALANCED|>'],
 'metrics': ['recall', 'f1score', 'precision', 'accuracy'],
 'values': ['70.12%', '48.54%', '37.12%', '81.13%'],
 'rates': ['MODERATE', 'LOW', 'LOW', 'MODERATE'],
 'narration': "The machine learning algorithm trained on this classification task was evaluated and it achieved a low F1score of 48.54% with a very low precision of 37.12% and a moderate recall (i.e. the prediction sensitivity) score of 70.12%. The accuracy score of 81.13% is not that impressive as the dummy model assigning the majority class #CA to any given input can achieve close to this performance. The model's overall classification performan

In [4]:
processInputTableAndNarrations = composePreambleAndInputs


In [5]:
test_sample = []
eval_tables = []
for pc in test_data:
    test_sample.append(processInputTableAndNarrations(pc,identical_metrics=identicals))
    #eval_tables.append(parseTableStructureForEval(pc,identicals))
rtest_sample = []
reval_tables = []
for pc in test_data:
    rtest_sample.append(processInputTableAndNarrations(pc,identical_metrics=identicals))
    #reval_tables.append(parseTableStructureForEval(pc,identicals))

In [6]:
len(full_data)

700

In [7]:
mask_met=False
processed=[]
preambles = []
# apply the preprocessing to the data
for idx,pc in enumerate(full_data):
    #print(idx)
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=False,)
    preambles.append(examples['preamble'])
    processed.append(examples)
    
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if examples['preamble'] not in set(preambles):
        preambles.append(examples['preamble'])
        processed.append(examples)
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if examples['preamble'] not in set(preambles):
        preambles.append(examples['preamble'])
        processed.append(examples)
        
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if examples['preamble'] not in set(preambles):
        preambles.append(examples['preamble'])
        processed.append(examples)

for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if len(examples['classes'])> 2:
        if examples['preamble'] not in set(preambles):
            preambles.append(examples['preamble'])
            processed.append(examples)

random.shuffle(processed)
random.shuffle(processed)

In [8]:
len(processed)

2301

In [1]:
import pickle as pk

In [2]:
# Save the dataset
#pk.dump(processed,open('../dataset/train_dataset.dat','wb'))
processed = pk.load(open('../dataset/train_dataset.dat','rb'))

In [4]:
len(processed)

2301

In [8]:
processed[13]

{'preamble': '<MetricsInfo> precision | VALUE_MODERATE | 60.32% <|> accuracy | VALUE_HIGH | 63.97% <|> f1score | VALUE_MODERATE | 60.8%  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|BALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB'],
 'dataset_attribute': ['<|BALANCED|>'],
 'metrics': ['f1score', 'precision', 'accuracy'],
 'values': ['60.8%', '60.32%', '63.97%'],
 'rates': ['MODERATE', 'MODERATE', 'HIGH'],
 'narration': 'The F1score, accuracy and precision are 60.8%, 63.97% and 60.32%, respectively. The given F1score and accuracy scoring is indicative of a model with fairly good signs of being accurate and precises in determining #CA and #CB. However, the models only perform decently well, with still room for improvement, and with similar precision and accuracy scores suggesting a combined issue with the model.'}